# Week 12.2.2  - Generating word embeddings using Gensim Word2Vec
## Word Embeddings and Word2Vec

In this notebook, rather than loading word embeddings trained on large generic datasets, we train our own embedding models.

![alt text](https://cdn.analyticsvidhya.com/wp-content/uploads/2017/06/06062705/Word-Vectors.png)

[analyticsvidhya](https://cdn.analyticsvidhya.com/wp-content/uploads/2017/06/06062705/Word-Vectors.png) 

## Dataset

This notebook is based on the Kavita Ganesan's tutorial, 

http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/

with the dataset `reviews_data.txt.gz` available at: 

https://github.com/kavgan/data-science-tutorials/tree/master/word2vec

The dataset is a single text file concatenated from 259K full hote reviews from OpinRank. The full OpinRank dataset also has car reviews, which is available at the UCI Machine Learning Repository.

http://archive.ics.uci.edu/ml/datasets/opinrank+review+dataset

#### Hotel Reviews

- Full reviews of hotels in 10 different cities (Dubai, Beijing, London, New York City, New Delhi, San Francisco, Shanghai, Montreal, Las Vegas, Chicago) 
- There are about 80-700 hotels in each city 
- Extracted fields include date, review title and the full review 
- Total number of reviews: ~259,000

In [ ]:
# Import required packages and setup logging functionality
import bokeh
import gzip
import gensim 
import logging

In [ ]:
# The data_file path will be different depending on where you've copied the notebooks...
data_file='../data/reviews_data.txt.gz'

### Read hotel reviews data

The original format of the hotel review data is in .zip which is composed of numerous .json data files, these are read into memory in iteratively. Furthermore, a gensim utility function `gensim.utils.simple_preprocess()` is used to pre-process the text as it's read.

`simple_preprocess()` converts a document (in this case a review) into a list of tokens that are lower cased.

The documents may take a minute or two to process

In [ ]:
def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    with gzip.open(input_file, 'rb') as f:
        for i, line in enumerate(f): 
            # Do some pre-processing and return a list of words for each review text 
            yield gensim.utils.simple_preprocess(line)

# Read the tokenized reviews into a list each review item becomes a series of words so this becomes a list of lists
documents = list(read_input(data_file))

## Understanding some of the parameters

To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model:
`model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)`

#### Parameters of Interest
`size`<br>
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. Typical sizes are 100-300. A value of 100-150 has worked well for me. 

`window`<br>
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window.

`min_count`<br>
Minimium frequency count of words. The model would ignore words that do not statisfy the min_count. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

`workers`<br>
How many threads to use behind the scenes?

Instead of training the word embeddings in this notebook, we can load ones that have already been trained for you!

In [1]:
from pathlib import Path
modelpath = Path('../data/word2vec/word2vec_reviews.bin').resolve()

In [2]:
model = gensim.models.Word2Vec.load(str(modelpath))

NameError: name 'gensim' is not defined

# Looking at outputs
- How are common words representated as vectors?
- Can we find similarities between words?

### Words represented as vectors

In [ ]:
# Get vector representation of a word
testWord = 'dirty'
print(f'Word - {testWord}\nVector Representation\n{model.wv.get_vector(testWord)}')
print(f'Shape of vector: {model.wv.get_vector(testWord).shape[0]}')

### Finding similarities between words

In [ ]:
w1 = "dirty"
model.wv.most_similar(positive=w1)

In [ ]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar(positive=w1,topn=6)

In [ ]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar(positive=w1,topn=6)

In [ ]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar(positive=w1,topn=6)

In [ ]:
# Get everything related to stuff on the bed
w1 = ['bed','sheet','pillow']
w2 = ['couch']
model.wv.most_similar(positive=w1,negative=w2,topn=10)

In [ ]:
# Similarity between two different words
print(f'Similarity between the words dirty and smelly: {model.wv.similarity(w1="dirty",w2="smelly"):0.3}')

In [ ]:
# Similarity between two identical words
print(f'Similarity between the same word dirty: {model.wv.similarity(w1="dirty",w2="dirty"):0.3}')

In [ ]:
# similarity between two unrelated words
print(f'Similarity between the words dirty and clean: {model.wv.similarity(w1="dirty",w2="clean"):0.3}')

In [ ]:
# Which one is the odd one out in this list?
oddOneOutList = ["cat", "dog", "france"]
print(f'Which words doesnt belong in the set [cat, dog, france]? Odd one out: {model.wv.doesnt_match(oddOneOutList)}')

In [ ]:
# Which one is the odd one out in this list?
oddOneOutList = ["bed","pillow","duvet","shower"]
print(f'Which words doesnt belong in the set [bed, pillow, duvet, shower]? Odd one out: {model.wv.doesnt_match(oddOneOutList)}')

## Visualising the word vectors in 2D space

Here we use a dimensionality reduction and visualisation package from Scikit-Learn, t-Distributed Stochastic Neighbor Embedding (t-SNE), which is particularly suited for visualising high dimensional data. Another popular options for dimensionality reduction is Principal Component Analysis (PCA).

Note: the dimension of our word vectors by choice is 150, typical numbers could be 50, 100, and 300. Among these numbers, a dimension of 300 has been shown as most effective in capturing the syntatic and semantic information of a word. However, it will take much longer to train. 

In [ ]:
# Load required statistical package
from sklearn.manifold import TSNE

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
# Need the interactive Tools for Matplotlib
%matplotlib inline
# Plot formatting
plt.rcParams["figure.figsize"] = [16,9]
font = {'size':16}
matplotlib.rc('font', **font)

In [ ]:
def display_closestwords_tsnescatterplot(model, word):
    
    arr = np.empty((0,150), dtype='f')
    word_labels = [word]

    # get close words
    close_words = model.similar_by_word(word)
    
    # add the vector for each of the closest words to the array
    arr = np.append(arr, np.array([model[word]]), axis=0)
    for wrd_score in close_words:
        wrd_vector = model[wrd_score[0]]
        word_labels.append(wrd_score[0])
        arr = np.append(arr, np.array([wrd_vector]), axis=0)
        
    # find tsne coords for 2 dimensions
    tsne = TSNE(n_components=2, random_state=0)
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)

    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    # display scatter plot
    plt.scatter(x_coords, y_coords)

    for label, x, y in zip(word_labels, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
    plt.xlim(x_coords.min()+0.00005, x_coords.max()+0.00005)
    plt.ylim(y_coords.min()+0.00005, y_coords.max()+0.00005)
    plt.title(f'Words closest to: {word}')
    plt.show()

### Visualising closest words in 2D

In [ ]:
display_closestwords_tsnescatterplot(model.wv, "bed")

# Generating word embeddings for domain-specific inforamtion
Word embeddings are interesting and thought provoking but
- What insights can we gain from them?
- How do we apply them to industry data?

This section of the notebook will explore word embeddings and their application to geological survey data (GSWA).

## Loading a small sample of Geological Survey of Western Australia (GSWA) data
Note: we have already added our Google Drive to the notebook, so we don't need to do this again.

In [ ]:
import zipfile
import json

In [ ]:
data_file="../data/wamex_xml.zip"

In [ ]:
# Import Dataset
data = list()
with zipfile.ZipFile(data_file, "r") as z:
    #df = [pd.read_json(filename) for filename in z.namelist()]
    print(len(z.namelist()))
    for filename in z.namelist():
        # print(filename)
        # df = pd.read_json(filename)
        with z.open(filename) as f:
            # load the json file
            # The resulting `content` is a list
            content = json.loads(f.read()) 
            # Convert content to a string   
            content = "".join(content)
            # Add to the data list
            data.append(content)

In [ ]:
# Previewing the data that we have loaded. This is very different to the hotel reviews dataset.
print(data[0])

In [ ]:
def read_input(input_file):
    """This method reads the input file which is in zip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))

    data = list()
    with zipfile.ZipFile(input_file, "r") as z:
    #df = [pd.read_json(filename) for filename in z.namelist()]
        print(len(z.namelist()))
        for i, filename in enumerate(z.namelist()):
            # print(filename)
            # df = pd.read_json(filename)
            if (i%100==0):
                logging.info ("read {0} reports".format (i))
            with z.open(filename) as f:
                # load the json file
                # The resulting `content` is a list
                content = json.loads(f.read()) 
                # Convert content to a string   
                content = "".join(content)
                if len(content) >= 10:
                    # Add to the data list
                    yield gensim.utils.simple_preprocess (content)
                else:
                    logging.info("removed {0} because of small size".format (filename))

In [ ]:
# Read the tokenized reviews into a list each review item becomes a series of words so this becomes a list of lists
documents = list(read_input (data_file))
logging.info("Done reading data file")

In [ ]:
# Review the first documents top 25 words. See how they have been pre-processed and tokenized.
print(documents[0][:25])

### Training word embedding model off of domain-specific text.

Note: we are required to use bigrams to aid our model due to domain-specific terms.

In [ ]:
import nltk

Similar to the hotel reviews, we load pre-trained embeddings rather than training them due to time requirements.

In [ ]:
modelDS = gensim.models.Word2Vec.load('../data/word2vec/word2vec_gswa.bin')

## Looking at domain-specific outputs

In [ ]:
print(list(model.wv.vocab.keys())[:100])

In [ ]:
print(modelDS.wv['gold'])

In [ ]:
w1 = "gold"
modelDS.wv.most_similar(positive = w1)

In [ ]:
# look up top 6 words similar to 'polite'
w1 = ["iron"]
modelDS.wv.most_similar(positive = w1, topn = 6)

In [ ]:
# get everything related to stuff on the commodity
w1 = ["gold",'commodity','ore']
w2 = ['rock']
modelDS.wv.most_similar(positive = w1, negative = w2, topn = 10)

In [ ]:
# similarity between two different words
modelDS.wv.similarity(w1 = "gold", w2 = "ore")

In [ ]:
# similarity between two identical words
modelDS.wv.similarity(w1 = "gold", w2 = "gold")

In [ ]:
# similarity between two unrelated words
modelDS.wv.similarity(w1 = "gold", w2 = "rock")

In [ ]:
# Which one is the odd one out in this list?
modelDS.wv.doesnt_match(["gold", "rock", "copper"])

### Comparing our domain-specific embeddings with embeddings trained on a different dataset (news articles).
- What are the nearest terms to 'commodity', 'ore', 'rock', etc.

Unfortunately due to the size of pretrained word embedding models they have been omitted from this comparison. However, the code below shows how to load the Google news word2vec embedding model. This model is trained on 100 billion words and is 1.6GB in size (massive!).

In [ ]:
# Load Google's pre-trained Word2Vec model.
# 1.6GB trained over 100B words; dimension of 300. Cannot use in this notebook due to massive size and time required to load.

#import gensim.downloader as api
#wv = api.load('word2vec-google-news-300')

In [ ]:
# Define function to compare top-n similariries for a given word between two embedding models.
def compare_words(word, topn, model1, model2):
    similarWordsModel1 = model1.wv.most_similar(positive=word, topn=topn)
    similarWordsModel2 = model2.wv.most_similar(positive=word,topn=topn)

    print(f'Top {topn} words similar to {word}\n(format: n | model 1 | model 2 )\n')
    for n in range(topn):
        print(f'{n+1} |{similarWordsModel1[n][0]:<15} | {similarWordsModel2[n][0]:<15}')

In [ ]:
# Looking at the word 'commodity'
compare_words(word = 'commodity', topn = 5, model1 = model, model2 = modelDS)

In [ ]:
# Looking at the word 'ore'
compare_words(word = 'ore', topn = 5, model1 = model, model2 = modelDS)

In [ ]:
# Looking at the word 'rock'
compare_words(word = 'rock', topn = 5, model1 = model, model2 = modelDS)

### Visualising the domain-specific word vectors in 2D space

In [ ]:
def display_closestwords_tsnescatterplot(model, word):
    
    arr = np.empty((0,50), dtype='f')
    word_labels = [word]

    # get close words
    close_words = model.similar_by_word(word)
    
    # add the vector for each of the closest words to the array
    arr = np.append(arr, np.array([model[word]]), axis=0)
    for wrd_score in close_words:
        wrd_vector = model[wrd_score[0]]
        word_labels.append(wrd_score[0])
        arr = np.append(arr, np.array([wrd_vector]), axis=0)
        
    # find tsne coords for 2 dimensions
    tsne = TSNE(n_components=2, random_state=0)
    np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)

    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    # display scatter plot
    plt.scatter(x_coords, y_coords)

    for label, x, y in zip(word_labels, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords='offset points')
    plt.xlim(x_coords.min()+0.00005, x_coords.max()+0.00005)
    plt.ylim(y_coords.min()+0.00005, y_coords.max()+0.00005)
    plt.title(f'Words closest to: {word}')
    plt.show()

In [ ]:
display_closestwords_tsnescatterplot(model = modelDS.wv, word = "gold")

#### Visualising all of the words in the vocabulary

In [ ]:
tsne = TSNE(n_components=2)
print(type(modelDS.wv.vocab))
X = modelDS[modelDS.wv.vocab]
# Shape of our model before t-SNE
print(f'Shape of model before t-SNE: {X.shape}')

In [ ]:
# Fitting subset of data to t-SNE
points = 2500
X_limited = X[:points]
X_tsne = tsne.fit_transform(X_limited)

# Uncomment line below to fit entire model to t-SNE
# X_tsne = tsne.fit_transform(X)

plt.rcParams['figure.figsize'] = [10, 10]
plt.scatter(X_tsne[:, 0], X_tsne[:, 1])
plt.show()

#### Interactive Visualisation

Refer to https://www.datascience.com/resources/notebooks/word-embeddings-in-python, and also for ideas of incoporating POS and bigrams into word2vec training.

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, output_notebook
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, output_notebook
from bokeh.models import ColumnDataSource, LabelSet
import pandas as pd

In [ ]:
def interactive_tsne(text_labels, tsne_array):
    '''makes an interactive scatter plot with text labels for each point'''

    # Define a dataframe to be used by bokeh context
    bokeh_df = pd.DataFrame(tsne_array, text_labels, columns=['x','y'])
    bokeh_df['text_labels'] = bokeh_df.index

    # interactive controls to include to the plot
    TOOLS="hover, zoom_in, zoom_out, box_zoom, undo, redo, reset, box_select"

    p = figure(tools=TOOLS, plot_width=700, plot_height=700)

    # define data source for the plot
    source = ColumnDataSource(bokeh_df)

    # scatter plot
    p.scatter('x', 'y', source=source, fill_alpha=0.6,
              fill_color="#8724B5",
              line_color=None)

    # text labels
    labels = LabelSet(x='x', y='y', text='text_labels', y_offset=8,
                      text_font_size="8pt", text_color="#555555",
                      source=source, text_align='center')

    p.add_layout(labels)

    # show plot inline
    output_notebook()
    show(p)

Launch interactive visualisation of word embeddings

In [ ]:
interactive_tsne(list(modelDS.wv.vocab.keys())[:points], X_tsne)